In [ ]:
%pip install -qU pypdf

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "../../00-example_data/layout-parser-paper.pdf",
)

In [ ]:
docs = loader.load()
docs[0]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(docs)

In [ ]:
texts

In [ ]:
len(texts)